__Fake News Classifier Using Bidirectional LSTM__

In [1]:
import pandas as pd

df = pd.read_csv('train.csv')
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [2]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.shape

(20800, 5)

In [6]:
df = df.dropna()

In [8]:
df.shape

(18285, 5)

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
# get the independent features
x = df.drop('label',axis=1)

In [11]:
y = df['label']

In [12]:
# check wherher dataset is balanced or not
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [13]:
import tensorflow as tf

In [15]:
tf.__version__

'2.13.0'

In [30]:
from tensorflow import keras
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dense

In [17]:
# vocabulary_size
voc_size = 5000

In [18]:
messages = x.copy()

In [19]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [20]:
import re
import nltk
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emon1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
messages.reset_index(inplace=True)

In [27]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [28]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
print(onehot_repr)

[[1006, 3853, 1675, 4870, 4819, 4938, 2347, 2157, 1684, 4001], [3167, 808, 4375, 4850, 2377, 4946, 3184], [1672, 3535, 1402, 399], [816, 396, 731, 2493, 3789, 1889], [994, 2377, 4689, 4246, 4612, 2716, 2377, 4842, 1120, 4686], [3743, 829, 547, 1036, 2729, 4208, 3161, 4204, 4811, 2054, 253, 152, 1276, 2170, 3184], [1100, 4225, 3320, 3849, 254, 830, 2602, 3294, 1899, 1714, 3350], [3570, 4777, 535, 1821, 1270, 278, 4208, 1851, 1899, 1714, 3350], [3288, 1876, 4413, 1392, 1338, 297, 655, 4904, 4208, 2120], [1851, 4352, 3707, 3526, 4332, 43, 1254, 275], [4453, 4512, 290, 3500, 2446, 4479, 989, 582, 1477, 1004, 3697], [2493, 2146, 4819, 297, 4208, 1270], [819, 1898, 3503, 1703, 848, 4769, 1307, 2485, 4297], [3920, 284, 2041, 3708, 668, 4982, 3567, 1899, 1714, 3350], [3398, 1023, 4634, 3561, 2238, 1899, 1714, 3350], [3651, 3603, 2124, 3322, 2151, 2999, 519, 4978, 1948, 4224], [953, 4273, 808], [2243, 1892, 2714, 2084, 4208, 3511, 1266, 3184], [2839, 4671, 4375, 1722, 3211, 1860, 1028, 4891, 27

__Embedding Representation__

In [29]:
sen_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sen_length)
embedded_docs

array([[   0,    0,    0, ..., 2157, 1684, 4001],
       [   0,    0,    0, ..., 2377, 4946, 3184],
       [   0,    0,    0, ..., 3535, 1402,  399],
       ...,
       [   0,    0,    0, ..., 1899, 1714, 3350],
       [   0,    0,    0, ..., 3571, 2597, 1838],
       [   0,    0,    0, ..., 2706,  699,  417]])

In [33]:
# creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sen_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_2 (Bidirecti  (None, 200)               112800    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
import numpy as np

x_final = np.array(embedded_docs)
y_final = np.array(y)

In [35]:
# train test split
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [36]:
# model training
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 37s 71ms/step - loss: 0.2649 - accuracy: 0.8793 - val_loss: 0.1970 - val_accuracy: 0.9190
Epoch 2/10
383/383 [==============================] - 23s 60ms/step - loss: 0.1307 - accuracy: 0.9503 - val_loss: 0.2067 - val_accuracy: 0.9165
Epoch 3/10
383/383 [==============================] - 17s 44ms/step - loss: 0.0840 - accuracy: 0.9696 - val_loss: 0.2225 - val_accuracy: 0.9165
Epoch 4/10
383/383 [==============================] - 13s 35ms/step - loss: 0.0474 - accuracy: 0.9847 - val_loss: 0.3480 - val_accuracy: 0.9150
Epoch 5/10
383/383 [==============================] - 15s 40ms/step - loss: 0.0346 - accuracy: 0.9881 - val_loss: 0.3785 - val_accuracy: 0.9069
Epoch 6/10
383/383 [==============================] - 16s 43ms/step - loss: 0.0211 - accuracy: 0.9938 - val_loss: 0.4299 - val_accuracy: 0.9102
Epoch 7/10
383/383 [==============================] - 18s 47ms/step - loss: 0.0148 - accuracy: 0.9956 - val_loss: 0.5930 - val_accuracy:

In [37]:
y_pred = model.predict(x_test)

189/189 [==============================] - 2s 9ms/step


In [38]:
y_pred

array([[9.9999762e-01],
       [4.0613741e-06],
       [8.5330714e-05],
       ...,
       [1.1060698e-07],
       [9.9985635e-01],
       [9.9999887e-01]], dtype=float32)

In [39]:
np.where(y_pred >= 0.5,1,0)

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [43]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [44]:
print(accuracy_score(y_test,y_pred))

ValueError: Classification metrics can't handle a mix of binary and continuous targets